In [ ]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ped_path = '/home/flutz/PycharmProjects/social-force-model/evaluation/vci-dataset-citr-master/data/trajectories_filtered/vci_front/front_interaction_01_traj_ped_filtered.csv'
veh_path = '/home/flutz/PycharmProjects/social-force-model/evaluation/vci-dataset-citr-master/data/trajectories_filtered/vci_front/front_interaction_01_traj_veh_filtered.csv'


In [ ]:
ped_df = pd.read_csv(ped_path)
ped_df

In [ ]:
ped_df.rename(columns={'id': 'ped_id', 'x_est': 'x', 'y_est': 'y', 'vx_est': 'v_x', 'vy_est': 'v_y'}, inplace=True)
ped_df['frame'] = ped_df['frame'] - ped_df['frame'].min()
# ped_df['y'] = ped_df['y']*-1
# ped_df['v_y'] = ped_df['v_y']*-1
ped_df.sort_values(by=['ped_id', 'frame'], inplace=True)
ped_df

In [ ]:
new_origin_x = ped_df['x'].values[0]
new_origin_y = ped_df['y'].values[0]
ped_df['x'] = ped_df['x'] - new_origin_x
ped_df['y'] = ped_df['y'] - new_origin_y

ped_df

In [ ]:
carla_origin_x = -50.0
carla_origin_y = 25.0

ped_df['x'] = ped_df['x'] + carla_origin_x
ped_df['y'] = ped_df['y'] + carla_origin_y
ped_df

In [ ]:
for key, grp in ped_df.groupby(['ped_id']):
    spawn_location = [grp['x'].iloc[0], grp['y'].iloc[0], 1.0]
    destination = [grp['x'].iloc[-1], grp['y'].iloc[-1], 0.0]
    speed = np.median(np.linalg.norm(grp[['v_x', 'v_y']].to_numpy(), axis=-1))

    print('[[walker.ped_spawner]]')
    print('blueprint = \'walker.pedestrian.0002\'')
    print('spawn_location = ' + str(spawn_location))
    print('destination = ' + str(destination))
    print('speed = ' + str(speed))
    print('quantity = 1')
    print('spawn_time = 0.0')
    print('')

In [ ]:
veh_df = pd.read_csv(veh_path)
veh_df.rename(columns={'id': 'veh_id', 'x_est': 'x', 'y_est': 'y', 'psi_est': 'heading', 'vel_est': 'vel'}, inplace=True)
veh_df['frame'] = veh_df['frame'] - veh_df['frame'].min()
# veh_df['y'] = veh_df['y']*-1
veh_df.sort_values(by=['veh_id', 'frame'], inplace=True)

veh_df.loc[veh_df.groupby('veh_id')['vel'].tail(3).index, ['vel']] = 0.0

veh_df

In [ ]:
veh_df['x'] = veh_df['x'] - new_origin_x + carla_origin_x
veh_df['y'] = veh_df['y'] - new_origin_y + carla_origin_y
veh_df

In [ ]:
for key, grp in veh_df.groupby(['veh_id']):
    trajectory = list(map(list, zip(grp['x'].to_list(), grp['y'].to_list())))
    headings = grp['heading'].to_list()
    speed = grp['vel'].to_list()
    
    print('[[vehicle.vehicle_spawner]]')
    print('auto_pilot = false')
    print('blueprint = \'vehicle.micro.microlino\'')
    print('trajectory = ' + str(trajectory))
    print('headings = ' + str(headings))
    print('speeds = ' + str(speed))
    print('quantity = 1')
    print('spawn_time = 0.0')
    print('')

    